In [1]:
from timeseries import *
display(HTML("<style>.container {width:99% !important; }</style>"))

In [2]:
print('fastai2    :', fastai2.__version__)
print('fastcore   :', fastcore.__version__)
print('torch      :', torch.__version__)
print('scipy      :', sp.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
print(f'Total RAM  : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Used RAM   : {bytes2GB(psutil.virtual_memory().used):5.2f} GB')
print('n_cpus     :', cpus)
iscuda = torch.cuda.is_available()
if iscuda: print('device     : {} ({})'.format(device, torch.cuda.get_device_name(0)))
else: print('device     :', device)

fastai2    : 0.0.16
fastcore   : 0.1.17
torch      : 1.4.0
scipy      : 1.3.1
numpy      : 1.18.1
pandas     : 0.25.1
Total RAM  :  8.00 GB
Used RAM   :  3.73 GB
n_cpus     : 8
device     : cpu


In [24]:
X_on_disk = np.load('./data/X_on_disk.npy', mmap_mode='r')
y_on_disk = np.load('./data/y_on_disk.npy', mmap_mode='r')
X_in_memory = np.load('./data/X_on_disk_small.npy')
y_in_memory = np.load('./data/y_on_disk_small.npy')
y = np.random.randint(0,10,len(X_on_disk))
X_on_disk.shape, y_on_disk.shape

((100000, 50, 512), (100000,))

In [25]:
train_len = int(len(X_on_disk)*.7)
splits = (L(list(np.arange(train_len))), L(list(np.arange(train_len, len(X_on_disk)))))

In [26]:
bs = 128
train_idx = np.random.randint(len(splits[0]), size=bs)
valid_idx = np.random.randint(len(splits[1]), size=bs)

In [27]:
tuple((TSTensor(X_on_disk[splits[0][train_idx]]), TensorCategory(y[splits[0][train_idx]])))

(TSTensor(samples:128, vars:50, len:512),
 TensorCategory([9, 4, 3, 7, 2, 8, 5, 8, 3, 3, 7, 1, 7, 1, 6, 5, 4, 9, 0, 2, 0, 1, 6, 7,
         2, 0, 5, 7, 7, 1, 6, 6, 9, 9, 5, 4, 5, 3, 8, 9, 6, 7, 2, 9, 7, 1, 9, 8,
         7, 1, 8, 6, 9, 3, 5, 0, 1, 9, 5, 0, 5, 1, 2, 4, 4, 4, 4, 7, 8, 0, 6, 7,
         9, 9, 0, 4, 0, 8, 3, 4, 6, 7, 5, 6, 5, 7, 8, 0, 1, 0, 8, 0, 8, 3, 2, 9,
         4, 1, 5, 8, 2, 8, 7, 9, 2, 6, 9, 6, 1, 8, 0, 6, 2, 2, 5, 8, 1, 7, 7, 8,
         1, 1, 1, 6, 7, 5, 7, 5]))

In [28]:
%timeit tuple((TSTensor(X_on_disk[splits[0][train_idx]]), TensorCategory(y[splits[0][train_idx]])))

8.26 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X_on_disk, y_on_disk, tfms=tfms, splits=splits, inplace=True)

In [30]:
train_ds, valid_ds = dsets.train, dsets.valid

In [31]:
train_ds.items[0][0]

memmap([[0.2626543 , 0.81090355, 0.32464218, ..., 0.23839498, 0.67757344,
         0.3310132 ],
        [0.7845197 , 0.68123066, 0.38421404, ..., 0.5383686 , 0.01147997,
         0.13235402],
        [0.6679299 , 0.56707335, 0.02308166, ..., 0.77351165, 0.20462573,
         0.9898304 ],
        ...,
        [0.38630033, 0.5770135 , 0.18095827, ..., 0.9798217 , 0.56657016,
         0.6018766 ],
        [0.09030128, 0.01747155, 0.5882474 , ..., 0.28344667, 0.6954193 ,
         0.93599343],
        [0.91863537, 0.01252496, 0.4146458 , ..., 0.80348647, 0.9133351 ,
         0.5418618 ]], dtype=float32)

In [32]:
train_ds[train_idx]

(TSTensor(samples:128, vars:50, len:512),
 TensorCategory([0, 2, 2, 8, 6, 6, 4, 0, 4, 5, 0, 8, 0, 5, 8, 3, 0, 5, 0, 8, 5, 3, 4, 1,
         7, 8, 3, 0, 6, 5, 5, 8, 5, 8, 1, 9, 7, 3, 3, 3, 3, 6, 4, 7, 2, 8, 7, 1,
         4, 4, 3, 9, 8, 2, 5, 0, 3, 4, 0, 6, 9, 5, 9, 8, 7, 4, 3, 0, 7, 9, 6, 3,
         3, 8, 2, 1, 1, 0, 0, 4, 9, 2, 8, 9, 9, 9, 8, 7, 6, 1, 5, 6, 4, 3, 2, 0,
         5, 1, 3, 5, 7, 8, 4, 0, 1, 6, 5, 8, 6, 8, 6, 3, 9, 8, 3, 6, 1, 4, 8, 5,
         1, 5, 4, 9, 8, 8, 7, 3]))

In [33]:
%timeit train_ds[train_idx]

11.2 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
dls = TSDataLoaders.from_dsets(train_ds, valid_ds, bs=[64, 128], num_workers=0)

In [46]:
b = dls.train.one_batch()
b

(TSTensor(samples:64, vars:50, len:512),
 TensorCategory([6, 1, 3, 1, 1, 8, 3, 1, 8, 9, 4, 8, 1, 5, 7, 3, 2, 2, 1, 3, 1, 5, 4, 8,
         0, 5, 9, 2, 3, 3, 4, 9, 0, 1, 2, 1, 0, 2, 5, 8, 7, 9, 0, 8, 1, 8, 7, 0,
         1, 3, 0, 0, 2, 6, 2, 2, 5, 5, 5, 8, 3, 7, 7, 7]))

In [36]:
%timeit cycle_dl(dls.train)

1min ± 9.06 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit cycle_dl(dls.train)

In [39]:
b = dls.train.one_batch()
b

(TSTensor(samples:64, vars:50, len:512),
 TensorCategory([3, 6, 6, 3, 8, 4, 7, 2, 3, 9, 4, 4, 6, 6, 0, 4, 0, 7, 4, 7, 0, 6, 6, 8,
         7, 9, 0, 9, 0, 6, 0, 5, 4, 0, 8, 3, 9, 3, 4, 2, 2, 2, 1, 4, 8, 2, 5, 2,
         5, 3, 9, 0, 7, 8, 7, 4, 6, 2, 7, 2, 0, 0, 5, 4]))

In [ ]:
# %timeit cycle_dl(dls.train)

In [34]:
self = train_ds
it = train_idx
%timeit tuple([typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
# %timeit tuple([typ(ptl[it])[...,self.sel_vars, self.sel_steps] if i==0 else typ(ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])

26.2 ms ± 469 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
TSTensor(self.ptls[0][train_idx])[...,self.sel_vars, self.sel_steps]

TSTensor(samples:128, vars:50, len:512)

In [20]:
self.types

[timeseries.core.TSTensor, fastai2.torch_core.TensorCategory]

In [12]:
dls   = TSDataLoaders.from_dsets(train_ds, valid_ds, bs=[64,128], num_workers=0)

In [13]:
train_dl, valid_dl = dls.train, dls.valid

In [14]:
b=next(iter(train_dl))
b

TypeError: int() argument must be a string, a bytes-like object or a number, not 'ellipsis'

In [12]:
%timeit cycle_dl(dls.train)

1min 10s ± 19.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
class NumpyDatasets(Datasets):
    "A dataset that creates tuples from X (and y) and applies `item_tfms`"
    _xtype, _ytype = NumpyTensor, None # Expected X and y output types (must have a show method)
    def __init__(self, X=None, y=None, items=None, tfms=None, tls=None, n_inp=None, dl_type=None, inplace=False, **kwargs):
        self.inplace = inplace
        if tls is None: 
            X = itemify(X, tup_id=0)
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X)) if y is None else tuple((X,y))
        self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        self.ptls = L([tl[:] if self.inplace else tl for tl,tfm in zip(self.tls, self.tfms)])
        if len(self.tls[0]) > 0: 
            self.types = [ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) for tl,_typ in zip(self.tls, [self._xtype, self._ytype])]
    
    def __getitem__(self, it):
        return tuple([typ(ptl[it] if i==0 else ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
    
    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, inplace=self.inplace, tfms=self.tfms)
    
    def _new(self, X, *args, y=None, **kwargs): 
        items = ifnoneelse(y,tuple((X,)),tuple((X, y)))
        return super()._new(items, tfms=self.tfms, do_setup=False, **kwargs)
    
    def show_at(self, idx, **kwargs):
        self.show(self[idx], **kwargs)
        plt.show()

    @property
    def items(self): return tuple([tl.items for tl in self.tls])
    @items.setter
    def items(self, vs):
        for tl,c in zip(self.tls, vs): tl.items = v


class TSDatasets(NumpyDatasets):
    "A dataset that creates tuples from X (and y) and applies `item_tfms`"
    _xtype, _ytype = TSTensor, None # Expected X and y output types (torch.Tensor - default - or subclass)
    def __init__(self, X=None, y=None, items=None, sel_vars=None, sel_steps=None, tfms=None, tls=None, n_inp=None, dl_type=None, 
                 inplace=False, **kwargs):
        self.inplace = inplace
        if tls is None: 
            X = itemify(X, tup_id=0)
            y = itemify(y, tup_id=0) if y is not None else y
            items = tuple((X)) if y is None else tuple((X,y))
        self.tfms = L(ifnone(tfms,[None]*len(ifnone(tls,items))))
        self.sel_vars = ifnone(sel_vars, slice(None))
        self.sel_steps = ifnone(sel_steps,slice(None))
        self.tls = L(tls if tls else [TfmdLists(item, t, **kwargs) for item,t in zip(items,self.tfms)])
        self.n_inp = (1 if len(self.tls)==1 else len(self.tls)-1) if n_inp is None else n_inp
        self.ptls = L([tl[:] if self.inplace else tl for tl,tfm in zip(self.tls, self.tfms)])
        if len(self.tls[0]) > 0: 
            self.types = [ifnone(_typ, type(tl[0]) if isinstance(tl[0], torch.Tensor) else tensor) for tl,_typ in zip(self.tls, [self._xtype, self._ytype])]
    
    def __getitem__(self, it):
        return tuple([typ(ptl[it][...,self.sel_vars, self.sel_steps] if i==0 else ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])
    
    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp, 
                                           inplace=self.inplace, tfms=self.tfms, sel_vars=self.sel_vars, sel_steps=self.sel_steps)
    @property
    def vars(self): return self[0][0].shape[-2]
    @property
    def length(self): return self[0][0].shape[-1]

In [10]:
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X_on_disk, y_on_disk, tfms=tfms, inplace=True)
# dls   = TSDataLoaders.from_dsets(dsets, bs=[64,128])

In [12]:
dls   = NumpyDataLoaders.from_dsets(dsets, bs=128)

NameError: name 'self' is not defined

In [8]:
# train_ds = dsets.train
# valid_ds = dsets.valid
train_dl = TSDataLoader(dsets.train, bs=64, shuffle=True, drop_last=True)
valid_dl = TSDataLoader(dsets.valid, bs=128)

ValueError: need at least one array to stack

In [27]:
b=next(iter(train_dl))
b

(NumpyTensor(shape:[64, 50, 512]),
 TensorCategory([2, 8, 4, 3, 2, 1, 1, 8, 9, 2, 4, 7, 1, 3, 7, 3, 6, 9, 6, 8, 4, 1, 4, 8,
         2, 2, 0, 8, 7, 6, 6, 8, 1, 9, 2, 1, 0, 2, 7, 0, 8, 4, 8, 0, 9, 6, 2, 3,
         4, 9, 0, 6, 5, 9, 8, 1, 8, 5, 6, 1, 7, 9, 1, 3]))

In [13]:
self = dsets
it = valid_idx

tuple([typ(ptl[it] if i==0 else ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])

(TSTensor(samples:128, vars:50, len:512),
 TensorCategory([8, 5, 7, 6, 2, 3, 9, 7, 7, 3, 4, 2, 7, 2, 1, 2, 0, 6, 2, 6, 4, 7, 5, 7,
         1, 0, 8, 7, 9, 7, 0, 9, 4, 7, 8, 8, 9, 4, 6, 5, 3, 8, 6, 6, 7, 4, 1, 2,
         9, 7, 4, 5, 0, 3, 1, 4, 0, 9, 8, 3, 5, 9, 3, 5, 3, 2, 4, 6, 4, 3, 1, 9,
         1, 4, 3, 6, 3, 5, 2, 9, 2, 3, 2, 4, 2, 0, 3, 4, 3, 2, 2, 6, 9, 2, 3, 1,
         2, 3, 7, 0, 1, 6, 3, 0, 3, 2, 4, 9, 6, 7, 9, 9, 9, 2, 4, 5, 3, 2, 5, 7,
         7, 8, 8, 4, 0, 3, 0, 2]))

In [14]:
%timeit tuple([ptl[it] for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])

330 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit tuple([typ(ptl[it] if i==0 else ptl[it]) for i,(ptl,typ) in enumerate(zip(self.ptls,self.types))])

11.3 ms ± 60.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
self.types

[timeseries.core.NumpyTensor, fastai2.torch_core.TensorCategory]

In [16]:
%timeit TSTensor(self.ptls[0][it])

10.1 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit TensorCategory(self.ptls[1][it])

1.33 ms ± 9.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%timeit TensorCategory(self.ptls[1][it])

In [41]:
mmap = self.ptls[0][it]

In [48]:
%timeit TSTensor(np.stack(mmap))

9.53 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [51]:
%timeit tensor(mmap)

8.41 s ± 1.37 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%timeit tensor(np.stack(mmap))

12.4 ms ± 2.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
isinstance(mmap, (list, L))

True

In [ ]:
dsets[]